<a href="https://colab.research.google.com/github/VARUN-OFFICIAL-24/Quanta/blob/main/Quanta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REQUIREMENTS




In [2]:
pip install pennylane torch torch-geometric networkx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 118.1 MB/s eta 0:00:00


# **INSTALL & GLOBAL CONFIG**

In [17]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import networkx as nx

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool


# ---------------- CONFIG ----------------

In [18]:
N_QUBITS = 5
MAX_DEPTH = 12

INIT_CIRCUITS = 15
BO_ITERS = 30
CANDIDATES_PER_ITER = 6

SURROGATE_EPOCHS = 60
MC_SAMPLES = 6
GATE_SET = ["RX", "RY", "RZ", "CNOT", "H"]

**Quantum Device**

In [19]:
dev = qml.device("default.qubit", wires=N_QUBITS)


**HARDER Objective**

In [20]:
def quantum_objective(expvals):
    score = 0.0
    for i in range(len(expvals)-1):
        score += (expvals[i] * expvals[i+1])**2
    score += 0.5 * np.std(expvals)
    return score


**Circuit Execution**

In [21]:
def execute_circuit(circuit, params):
    @qml.qnode(dev)
    def qc():
        for gate, wires, idx in circuit:
            if gate == "RX":
                qml.RX(params[idx], wires=wires)
            elif gate == "RY":
                qml.RY(params[idx], wires=wires)
            elif gate == "RZ":
                qml.RZ(params[idx], wires=wires)
            elif gate == "H":
                qml.Hadamard(wires=wires)
            elif gate == "CNOT":
                qml.CNOT(wires=wires)
        return [qml.expval(qml.PauliZ(i)) for i in range(N_QUBITS)]
    return qc()


**Evaluation**

In [22]:
def evaluate_circuit(circuit):
    params = np.random.uniform(0, np.pi, len(circuit))
    try:
        expvals = execute_circuit(circuit, params)
        perf = quantum_objective(expvals)
        cost = 0.05 * len(circuit)
        return perf - cost
    except:
        return -10.0


 **Random Circuit Generator**

In [23]:
def random_circuit():
    circuit = []
    depth = random.randint(4, MAX_DEPTH)

    for i in range(depth):
        gate = random.choice(GATE_SET)
        if gate == "CNOT":
            wires = random.sample(range(N_QUBITS), 2)
        else:
            wires = [random.randint(0, N_QUBITS - 1)]
        circuit.append((gate, wires, i))

    return circuit


**Circuit → Graph**

In [24]:
def circuit_to_graph(circuit):
    G = nx.DiGraph()
    for i, (gate, _, _) in enumerate(circuit):
        G.add_node(i, gate=gate)
        if i > 0:
            G.add_edge(i-1, i)

    x = []
    for _, d in G.nodes(data=True):
        x.append([1 if d["gate"] == g else 0 for g in GATE_SET])

    x = torch.tensor(x, dtype=torch.float)
    edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
    if edge_index.numel() == 0:
        edge_index = torch.zeros((2,1), dtype=torch.long)

    return Data(x=x, edge_index=edge_index)


**GNN Surrogate**

In [25]:
class GNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(len(GATE_SET), 64)
        self.conv2 = GCNConv(64, 64)
        self.fc = nn.Linear(64, 1)

    def forward(self, data):
        x = F.relu(self.conv1(data.x, data.edge_index))
        x = F.relu(self.conv2(x, data.edge_index))
        x = global_mean_pool(x, torch.zeros(x.size(0), dtype=torch.long))
        return self.fc(x)


In [ ]:
model = GNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

**Uncertainty**

In [26]:
def predict_with_uncertainty(graph):
    model.train()
    preds = [model(graph).item() for _ in range(MC_SAMPLES)]
    return np.mean(preds), np.std(preds)


**Acquisition Function**

In [27]:
def expected_improvement(mu, sigma, best):
    return (mu - best) * 0.6 + sigma


**STRONG Mutation**

In [28]:
def mutate(circuit, patterns=None):
    new = circuit.copy()

    if patterns and random.random() < 0.3:
        return random.choice(patterns)

    if random.random() < 0.4 and len(new) > 3:
        new.pop(random.randint(0, len(new)-1))

    if random.random() < 0.3 and len(new) > 3:
        i, j = random.sample(range(len(new)), 2)
        new[i], new[j] = new[j], new[i]

    if random.random() < 0.3:
        gate = random.choice(GATE_SET)
        wires = random.sample(range(N_QUBITS), 2) if gate=="CNOT" else [random.randint(0, N_QUBITS-1)]
        new.append((gate, wires, len(new)))

    return new


# **User Input Circuit (You can change it with your desired input)**

In [29]:
user_circuit = [
    # Superposition layer
    ("H", [0], 0),
    ("H", [1], 1),
    ("H", [2], 2),
    ("H", [3], 3),
    ("H", [4], 4),

    # Chain entanglement
    ("CNOT", [0, 1], 5),
    ("CNOT", [1, 2], 6),
    ("CNOT", [2, 3], 7),
    ("CNOT", [3, 4], 8),

    # Redundant / harmful entanglement
    ("CNOT", [0, 1], 9),   # redundant
    ("CNOT", [1, 2], 10),  # redundant

    # Over-parameterized rotations
    ("RX", [0], 11),
    ("RX", [1], 12),
    ("RX", [2], 13),
    ("RX", [3], 14),
    ("RX", [4], 15),
]


**Optimization Loop**

In [31]:


circuits = [user_circuit]
while len(circuits) < INIT_CIRCUITS:
    circuits.append(random_circuit())

scores = [evaluate_circuit(c) for c in circuits]
graphs = [circuit_to_graph(c) for c in circuits]

patterns = []

for it in range(BO_ITERS):

    model.train()
    for _ in range(SURROGATE_EPOCHS):
        for g, y in zip(graphs, scores):
            pred = model(g).squeeze()
            # Fix: Explicitly cast target tensor to torch.float32
            loss = loss_fn(pred, torch.tensor(y, dtype=torch.float32))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    best_so_far = max(scores)

    candidates = [mutate(random.choice(circuits), patterns)
                  for _ in range(CANDIDATES_PER_ITER)]

    ranked = []
    for c in candidates:
        mu, sigma = predict_with_uncertainty(circuit_to_graph(c))
        acq = expected_improvement(mu, sigma, best_so_far)
        ranked.append((acq, c))

    ranked.sort(reverse=True)
    best_candidate = ranked[0][1]

    score = evaluate_circuit(best_candidate)

    circuits.append(best_candidate)
    scores.append(score)
    graphs.append(circuit_to_graph(best_candidate))

    if score > np.percentile(scores, 80):
        patterns.append(best_candidate)

    print(f"Iteration {it:02d} | Score = {score:.4f}")

Iteration 00 | Score = -10.0000
Iteration 01 | Score = 1.2104
Iteration 02 | Score = 3.0103
Iteration 03 | Score = 4.0433
Iteration 04 | Score = 0.9946
Iteration 05 | Score = 0.9653
Iteration 06 | Score = 3.6422
Iteration 07 | Score = 1.4580
Iteration 08 | Score = 1.5829
Iteration 09 | Score = 0.9449
Iteration 10 | Score = 3.9728
Iteration 11 | Score = 3.4868
Iteration 12 | Score = 3.1051
Iteration 13 | Score = 3.2908
Iteration 14 | Score = 3.2777
Iteration 15 | Score = 4.1994
Iteration 16 | Score = 3.0685
Iteration 17 | Score = 3.4911
Iteration 18 | Score = 3.0016
Iteration 19 | Score = 3.8707
Iteration 20 | Score = 3.7415
Iteration 21 | Score = 2.9491
Iteration 22 | Score = 4.0815
Iteration 23 | Score = 2.8205
Iteration 24 | Score = 3.3429
Iteration 25 | Score = 3.0596
Iteration 26 | Score = 3.2378
Iteration 27 | Score = 1.0396
Iteration 28 | Score = 3.0350
Iteration 29 | Score = 3.8980


# **Final Output**

In [32]:
best_idx = np.argmax(scores)
best_circuit = circuits[best_idx]

print("\n===== USER CIRCUIT =====")
for gate in user_circuit:
    print(gate)

print("\n===== FINAL OPTIMIZED CIRCUIT =====")
for gate in best_circuit:
    print(gate)

print("\nFinal Score:", scores[best_idx])
print("Gate Count Before:", len(user_circuit))
print("Gate Count After :", len(best_circuit))



===== USER CIRCUIT =====
('H', [0], 0)
('H', [1], 1)
('H', [2], 2)
('H', [3], 3)
('H', [4], 4)
('CNOT', [0, 1], 5)
('CNOT', [1, 2], 6)
('CNOT', [2, 3], 7)
('CNOT', [3, 4], 8)
('CNOT', [0, 1], 9)
('CNOT', [1, 2], 10)
('RX', [0], 11)
('RX', [1], 12)
('RX', [2], 13)
('RX', [3], 14)
('RX', [4], 15)

===== FINAL OPTIMIZED CIRCUIT =====
('RZ', [4], 0)
('RY', [4], 1)
('CNOT', [3, 2], 2)
('RY', [4], 3)

Final Score: 4.199448256409773
Gate Count Before: 16
Gate Count After : 4
